### wtfPython docu

## 1.string interning

In [2]:
a="a_b"
b="a"+"_"+"b"
print(id(a)==id(b))

True


In [11]:
a=a+"!"
b=b+"!"
print(id(a) is id(b))

False


In [10]:
a=''.join(['f','o','o'])
b=''.join(['f','o','o'])
a is  b

False

In [12]:
a='a'*20
b='a'*21
a is b

False

Such behavior is due to Cython optimization called **`string interning`** that tries to use the existing immutable objects rather than creating a new object to save memory.
- all length 0 and 1 string are interned
- strings will be interned at compile time instead of run time 
- strings that are not composed of ASCII letters ,digits or underscores are not interned
> by the way **the different of 'id' and '=='**<br>
'id' checks if both the operands refer to the same object .it is for reference equality<br>
'==' compares the values of both the operands and checks if they are the same. it is for the value equality

## 2.peephole optimization  constant floding  small_ints cache

In [13]:
import dis
def foo():
    return 'foo'+'bar'
dis.dis(foo)

  3           0 LOAD_CONST               3 ('foobar')
              2 RETURN_VALUE


In [7]:
a,b=257,257
print(a==b)
print(a is b)
a=257
b=257
print(a==b)
print(a is b)

True
True
True
False


- source code will be transformed to **`raw byte code`** first than transformed to 'more efficient byte code ' via peephole optimization  
- **`constant folding`** only occurs strings having length less than 20 (wont let **`.pyc`** to large) to reduce few clock cycles during runtime  
- python will cache the frequency number to a list named **`small_ints`** just for **`[-5,256]`**  
' = ' assigning operator in the same line will allocate same memory if just one value for multi-variances 

In [33]:
hash(1.0) is hash(1)

True

python dict check for equality and compare `hash(key)` to determinate if two keys have same value

In [ ]:
import dis
def foo():
    a=1
    print(id(a))
    try:
        a+=1
        print(id(a))
        return a
    finally:
        a+=1
        print(id(a))

    return a
# print(dis.dis(foo))
print(foo())
b="1"
print(id(b))
b="1"
print(id(b))
print("---------------------")
c=[1,2]
print(c,' ',id(c[0]),' ',id(c[1]),' ',id(c))
c[0]=3
print(c,' ',id(c[0]),' ',id(c[1]),' ',id(c))
id(c)

不可变变量会根据值的内容分配空间 并且不能改变改空间的内容只能更改指向  
可变变量也是根据指针决定内存空间，但是直接指向的空间存的也是深层内容的指针

In [1]:
array=[1,8,15]
g=(i for i in array if array.count(i)>0)
array=[2,8,22]
print(list(g))

[8]


In　a `generator` expression `in` is declareation time to determinate some constant or others, but the `if` conditional clause is evaluated at runtime
SO `in`